# 1D Three Phase Simulation of Alloys and PINN model development 


This notebook contains the simulation of 1D Phase change of aluminium alloy. There will be three phases (solid,liquid and mushy).   

The approach used is finite difference method and the physics involved in heat conduction.

Import Libraries

In [2]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import csv
from sklearn import svm
import pandas as pd
import itertools

from ht_sim_2 import sim1d


from pyDOE2 import fullfact
from pyDOE2 import fracfact

import statsmodels.api as sm

from statsmodels.formula.api import ols
from scipy.stats import ttest_ind



### <center>Pareto analysis</center>

<p style="font-size:12px; font-family:'Times New Roman', sans-serif; line-height:1.6;">

In this section sensitivity study of the soldification is performed with respect to different material properties and  initial/boundary conditions. The soldification time is the variable under study and it is calcualted based on solution of 1D heat transfer and phase change equation.



Here the parameters/factors are as follows:-

1. Density for material in liquid state $\rho_l$.<br>
2. Density for material in solid state $\rho_s$.<br>
3. Latent Heat of Fusion <br>
4. Specific heat of material in liquid state $C_{pl}$ <br>
5. Specific heat of material in solid state $C_{ps}$ <br>
6. Thermal Conductivity of material in liquid state $k_l$<br>
7. Thermal Conductivity of material in solid state $k_s$ <br>


Boundary conditions:-
8. Surrounding Temperature <br>

Initial Conditions:-

9. Initial_temperature <br>


</p>




A full factorial DOE table is generated to study solidifcation time with the different factors and their corresponding levels.

In [3]:
# Create a full factorial design

num_levels = 2 
levels = [0, 1]
num_vars = 9
design = fullfact([num_levels]*num_vars)

# Create a DataFrame from the full factorial design

doe_df_sol_time = pd.DataFrame(design, columns=[f'Var_{i}' for i in range(1, num_vars+1)])
print(doe_df_sol_time.shape)

(512, 9)


The number of runs are large so a fractional factorial design is adopted.

In [4]:
#Create a fractional factorial design

from pyDOE2 import fracfact

num_levels = 2
levels = [0, 1]
num_vars = 9
design2 = fracfact('a b c d e  abcde abcd bcde adce ')
L_level = 0.99
R_level = 1.01
factor_levels = {
    'rho_l': [2760.0, 2860.0],
    'rho_s': [3000.0, 4000.0],
    'k_l': [96.0, 120.0],
    'k_s': [110.0, 130.0],
    'cp_l': [927.0, 947.0],
    'cp_s': [967.0, 987.0],
    'Surr_temp': [313.0, 323.0],
    'L_fusion': [389e3, 400e3 ],
    'temp_init': [880.0, 890.0],

}

factor_names = list(factor_levels.keys())

# Create a DataFrame from the fractional factorial design
doe_df_sol_time_fracfact = pd.DataFrame(design2, columns=factor_names)

for factor, levels in factor_levels.items():
    doe_df_sol_time_fracfact[factor] = doe_df_sol_time_fracfact[factor].map({-1: levels[0], 1: levels[1]})

    
print(doe_df_sol_time_fracfact.shape)

(32, 9)


Latin Hypercube Sampling is then explored.

In [5]:
n_factors = 9
n_levels = 2

# Create a Latin Hypercube Design

from pyDOE2 import lhs

design3 = lhs(n_factors, samples=100)
factor_levels = {
    'rho_l': [2760.0, 2761.0],
    'rho_s': [3000.0, 3001.0],
    'k_l': [96.0, 120.0],
    'k_s': [110.0, 130.0],
    'cp_l': [927.0, 947.0],
    'cp_s': [967.0, 987.0],
    'Surr_temp': [313.0, 323.0],
    'L_fusion': [389e3, 400e3 ],
    'temp_init': [880.0, 890.0],

}

factor_names = list(factor_levels.keys())
doe_lhs = pd.DataFrame(design3, columns=factor_names)

for i, (lower, upper) in enumerate(factor_levels.values()):
    doe_lhs.iloc[:, i] = lower + doe_lhs.iloc[:, i] * (upper - lower)



In [6]:
Lhs_doe_sol_time = doe_lhs.copy()

Lhs_doe_sol_time['total_sol_time'] = [0.0] * Lhs_doe_sol_time.shape[0]

for i in range(Lhs_doe_sol_time.shape[0]):
    input_values = Lhs_doe_sol_time.iloc[i,:-1].values
    Lhs_doe_sol_time.at[i, 'total_sol_time'] = sim1d(*input_values)

print(Lhs_doe_sol_time.shape)

TypeError: sim1d() missing 1 required positional argument: 'htc'

In [ ]:
plt.hist(Lhs_doe_sol_time['total_sol_time'], bins=10, alpha=0.5, color='b')
plt.title('Total Soldification time')
plt.xlabel('Seconds')
plt.ylabel('Frequency')
plt.show()

In [ ]:
formula = 'total_sol_time ~ C(rho_l) + C(rho_s) + C(k_l) + C(k_s) + C(cp_l) + C(cp_s) + C(Surr_temp) + C(L_fusion) + C(temp_init)'

model_lhs = sm.OLS.from_formula(formula, data=Lhs_doe_sol_time).fit()
residual_lhs_ols = model_lhs.resid
Lhs_doe_sol_time['Residuals'] = residual_lhs_ols
print(model_lhs.summary())

In [ ]:
from scipy.stats import shapiro

# Shapiro-Wilk Test
stat, p_value = shapiro(residual_lhs_ols)
print('Shapiro-Wilk Test Statistic:', stat)
print('p-value:', p_value)

if p_value > 0.05:
    print('The residuals are normally distributed (fail to reject H0).')
else:
    print('The residuals are not normally distributed (reject H0).')


# Plot residuals vs. fitted values
plt.scatter(model_lhs.fittedvalues, residual_lhs_ols)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs. Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:


# Create a Latin Hypercube Design

from scipy.stats import qmc

n_factors = 9
n_samples =200

design_sobol = qmc.Sobol(n_factors, scramble=True).random(n_samples)

# print(sobol_sample)

factor_levels = {
    'rho_l': [2760.0, 2761.0],
    'rho_s': [3000.0, 3001.0],
    'k_l': [96.0, 120.0],
    'k_s': [110.0, 130.0],
    'cp_l': [927.0, 947.0],
    'cp_s': [967.0, 987.0],
    'Surr_temp': [313.0, 323.0],
    'L_fusion': [389e3, 400e3 ],
    'temp_init': [880.0, 890.0],

}

factor_names = list(factor_levels.keys())
doe_sobol = pd.DataFrame(design_sobol, columns=factor_names)

for i, (lower, upper) in enumerate(factor_levels.values()):
    doe_sobol.iloc[:, i] = lower + doe_sobol.iloc[:, i] * (upper - lower)

sobol_doe_sol_time = doe_sobol.copy()

sobol_doe_sol_time['total_sol_time'] = [0.0] * sobol_doe_sol_time.shape[0]

for i in range(sobol_doe_sol_time.shape[0]):
    input_values = sobol_doe_sol_time.iloc[i,:-1].values
    sobol_doe_sol_time.at[i, 'total_sol_time'] = sim1d(*input_values)

print(sobol_doe_sol_time.shape)



In [ ]:
plt.hist(sobol_doe_sol_time['total_sol_time'], bins=10, alpha=0.5, color='b')
plt.title('Total Soldification time')
plt.xlabel('Seconds')
plt.ylabel('Frequency')
plt.show()

In [ ]:
formula = 'total_sol_time ~ C(rho_l) + C(rho_s) + C(k_l) + C(k_s) + C(cp_l) + C(cp_s) + C(Surr_temp) + C(L_fusion) + C(temp_init)'

model_sobol = sm.OLS.from_formula(formula, data=sobol_doe_sol_time).fit()
residual_sobol_ols = model_sobol.resid
sobol_doe_sol_time['Residuals'] = residual_sobol_ols
print(model_sobol.summary())

In [ ]:
from scipy.stats import shapiro

# Shapiro-Wilk Test
stat, p_value = shapiro(residual_sobol_ols)
print('Shapiro-Wilk Test Statistic:', stat)
print('p-value:', p_value)

if p_value > 0.05:
    print('The residuals are normally distributed (fail to reject H0).')
else:
    print('The residuals are not normally distributed (reject H0).')


# Plot residuals vs. fitted values
plt.scatter(model_sobol.fittedvalues, residual_sobol_ols)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs. Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()